In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
import tensorflow as tf
tf.enable_eager_execution()
print(tf.__version__)

1.15.0


In [3]:
import sys
from video.data import read_dataset, multi_hot, CLASS_NUM

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
eval_data_path = 'gs://asl-mixi-project-bucket/data/youtube-8m/valid/*.tfrecord'
batch_size = 128
ds = read_dataset(eval_data_path, tf.estimator.ModeKeys.EVAL, batch_size=batch_size)

In [5]:
prediction2 = multi_hot(list(range(2)), CLASS_NUM)
prediction3 = multi_hot(list(range(3)), CLASS_NUM)
prediction5 = multi_hot(list(range(5)), CLASS_NUM)
prediction10 = multi_hot(list(range(10)), CLASS_NUM)
prediction2

<tf.Tensor: id=77, shape=(3862,), dtype=float32, numpy=array([1., 1., 0., ..., 0., 0., 0.], dtype=float32)>

In [6]:
def get_recall(label, predict):
    return tf.reduce_mean(
        tf.reduce_sum(label * predict, axis=-1) / tf.reduce_sum(label, axis=-1)
    )
get_recall(
    label=tf.constant([[1, 1, 0, 0], [1, 1, 1, 0]]),
    predict=tf.constant([1, 1, 0, 0]),
)

<tf.Tensor: id=110, shape=(), dtype=float64, numpy=0.8333333333333333>

In [7]:
recall = 0.0
recall2 = 0.0
recall5 = 0.0
recall10 = 0.0

step = 100
for i, (feature, label) in enumerate(ds.take(step)):
    sys.stdout.write('\r{}'.format(i))
    label = label[:, :CLASS_NUM]
    recall += get_recall(label, prediction3) / step
    recall2 += get_recall(label, prediction2) / step
    recall5 += get_recall(label, prediction5) / step
    recall10 += get_recall(label, prediction10) / step

print('')
print('recall:', recall.numpy())
print('recall top 2:', recall2.numpy())
print('recall top 5:', recall5.numpy())
print('recall top 10:', recall10.numpy())


99
recall: 0.13280897
recall top 2: 0.1039775
recall top 5: 0.20871462
recall top 10: 0.31437227
